# Finding the Lost Robbies on Ethereum
## Treasure Hunting through Ethereum Event Logs


<table <tr> <td> <img src="https://sothebys-md.brightspotcdn.com/dims4/default/b3f1d33/2147483647/strip/true/crop/2000x1327+0+0/resize/800x531!/quality/90/?url=http%3A%2F%2Fsothebys-brightspot.s3.amazonaws.com%2Fmedia-desk%2F68%2F75%2F92068a3143a6ae4bb87ee8c64304%2Fl22235-c5p86-03-t1.jpg" width=800/></td>
<td> <img src="https://sothebys-md.brightspotcdn.com/dims4/default/5ff55bb/2147483647/strip/true/crop/2000x1323+0+0/resize/800x529!/quality/90/?url=http%3A%2F%2Fsothebys-brightspot.s3.amazonaws.com%2Fmedia-desk%2F3b%2F4f%2F38b59d5c4468aa5041e8dede6efb%2Fl22235-c5p86-04-t1.jpg" width=800/></td>
<td><img src="https://sothebys-md.brightspotcdn.com/dims4/default/38438a9/2147483647/strip/true/crop/1024x1024+0+0/resize/800x800!/quality/90/?url=http%3A%2F%2Fsothebys-brightspot.s3.amazonaws.com%2Fmedia-desk%2Fd6%2Fa6%2Fe9697f524b75864249642566bb94%2Fl22235-c5p86-cs-01.jpg" height=400/></td></tr></table>

Source: [Sotheby's](https://www.sothebys.com/en/buy/auction/2022/the-now-evening-auction/ai-generated-nude-portrait-7-frame-64)

## Blockchain Archaeology.

This notebook explores the data behind the legendary story of the "Lost Robbies": Robbie Barrat's series of 300 A.I. generated artworks minted on the SuperRare crypto art marketplace on Ethereum and handed out as gifts to attendees of the 2018 Christie's Art+Tech Summit. Today, Robbie's works are highly respected in the crypto art community and have sold for millions of dollars. Yet to this day, the vast majority of those works handed out in 2018 have never been claimed; the gift cards enscribing the private key to access the NFTs lying dormant on Ethereum.

How many of these treasured NFTs are actually lost and how can we tell from reading data from the Ethereum blockchain? In this notebook, I'll walk through how to interpret and pull some data from Ethereum tapping into Infura and web3 Python client. 

# Set Up

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, timezone
from web3 import Web3
from web3._utils.events import get_event_data
import json
import requests
import io
from PIL import Image,ImageOps
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img

To start, we need a way to access Ethereum blockchain data. At its core, Ethereum is a [distributed network](https://youtu.be/MlJPjJQZtC8?t=681) of computers known as nodes that run software for processing Ethereum transactions and verifying blocks. Some of these nodes (called full nodes) hold the entire blockchain's history.

Some enthusiasts run their own nodes and can access blockchain data via their personal node but given the technical complexity many analysts opt to use ["nodes as a service"](https://ethereum.org/en/developers/docs/nodes-and-clients/nodes-as-a-service/#top) providers that run optimized node infrastucture and build APIs that allow users to read and write to the Ethereum blockchain without having to run their own node. The pros of nodes as a service is that they allow for quicker access to data through familiar infrastucture. But it comes at a cost to decentralization: users must rely on a third party building in a point of centralization and dependence.

There are many different types of these services but below I'll be using Infura, which offers a free tier.

In [ ]:
#Connecting to Ethereum mainnet using Infura - check out https://infura.io/ to get a project ID
project_id = ''
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/{}'.format(project_id)))
if web3.isConnected() == True: print("Connected to Ethereum Mainnet")

# Step 1: Retrieving Transfer Event Logs

Every treasure hunt needs a map, and knowing what to look for on Ethereum starts with understanding smart contracts. Smart contracts are programs with functions and data stored on Ethereum. The main purpose of smart contracts is to execute decentralized application (dapp) program logic in a reliable manner. But another important component to Ethereum smart contracts are event logs. Event logs are like Ethereum's announcer, and tell us what is happening and changing on the field (blockchain). Developers include event logs that get emitted when certain interesting things happen on the blockchain like the creation / minting of a new NFT or ownership change / transfer of a token.

Most major NFT projects today follow the [ERC-721 standard](http://erc721.org/). SuperRare's original (v1) contract [0x41a322b28d0ff354040e2cbc676f0320d8c8850d](https://etherscan.io/address/0x41a322b28d0ff354040e2cbc676f0320d8c8850d#code) which Robbie's works were minted with
implements ERC721, which includes the `transfer` event. When an NFT is minted there is a `transfer` of the token to the artist from the zero address `0x000...000` bringing the NFT into existence. So we can search for SuperRare transfer events.

Below is an example showing the transfer event that created one of the lost robbies: Robbie's AI Generated Nude Portrait #7 Frame #1 (SR token 191).

In [ ]:
event_signature        = web3.keccak(text="Transfer(address,address,uint256)").hex()
token_contract_address = '0x41a322b28d0ff354040e2cbc676f0320d8c8850d'

# Get transfer events
logs = web3.eth.getLogs({
    "fromBlock": 5977236,
    "toBlock":   5977236,
    "address":   web3.toChecksumAddress(token_contract_address), #checksum
    "topics":    [event_signature]
})

In [ ]:
tx_event_abi = {
    'anonymous': False,
    'inputs': [
        {'indexed': True, 'name': 'from', 'type': 'address'},
        {'indexed': True, 'name': 'to', 'type': 'address'},
        {'indexed': False, 'name': 'tokenId', 'type': 'uint256'}],
    'name': 'Transfer',
    'type': 'event'
}

In [ ]:
print(logs[0]["transactionHash"].hex())

In [ ]:
tx = [get_event_data(web3.codec, tx_event_abi, log)["args"] for log in logs]

In [ ]:
print(f"Token {tx[0]['tokenId']} was sent from {tx[0]['from']} to {tx[0]['to']}")

abi = json.loads('[{"constant":false,"inputs":[{"name":"_uri","type":"string"},{"name":"_editions","type":"uint256"},{"name":"_salePrice","type":"uint256"}],"name":"addNewTokenWithEditions","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"},{"name":"_salePrice","type":"uint256"}],"name":"setSalePrice","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"_name","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_tokenId","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"currentBidDetailsOfToken","outputs":[{"name":"","type":"uint256"},{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"approvedFor","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"acceptBid","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_creator","type":"address"}],"name":"isWhitelisted","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"bid","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"tokensOf","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_percentage","type":"uint256"}],"name":"setMaintainerPercentage","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_creator","type":"address"}],"name":"whitelistCreator","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"ownerOf","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_uri","type":"string"}],"name":"originalTokenOfUri","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"_symbol","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"cancelBid","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"salePriceOfToken","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_tokenId","type":"uint256"}],"name":"transfer","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"takeOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_percentage","type":"uint256"}],"name":"setCreatorPercentage","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"tokenURI","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"creatorOfToken","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"buy","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_uri","type":"string"}],"name":"addNewToken","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"creatorPercentage","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"maintainerPercentage","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_creator","type":"address"}],"name":"WhitelistCreator","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"Bid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_seller","type":"address"},{"indexed":false,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"AcceptBid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"CancelBid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_buyer","type":"address"},{"indexed":true,"name":"_seller","type":"address"},{"indexed":false,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"Sold","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_tokenId","type":"uint256"},{"indexed":true,"name":"_price","type":"uint256"}],"name":"SalePriceSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"},{"indexed":true,"name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":true,"name":"_to","type":"address"},{"indexed":false,"name":"_tokenId","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_owner","type":"address"},{"indexed":true,"name":"_approved","type":"address"},{"indexed":false,"name":"_tokenId","type":"uint256"}],"name":"Approval","type":"event"}]')
contract = web3.eth.contract(address= web3.toChecksumAddress(token_contract_address), abi=abi)

def get_tokenURI(token_id):
    tokenURI = contract.functions.tokenURI(token_id).call()
    return tokenURI

def get_metadata(tokenURI):
    metadata = requests.get(tokenURI).json()
    return metadata

def load_image(metadata):
    r = requests.get(metadata['image'])
    i = Image.open(io.BytesIO(r.content))
    return i

load_image(get_metadata(get_tokenURI(tx[0]['tokenId'])))

In [ ]:
get_metadata(get_tokenURI(recent_tx[0]['tokenId']))

# Step 2: Getting all Transfer Events

Now that we know what to look for we can query the whole history and listen for SuperRare transfer events.

In [ ]:
def get_transfer_events(web3, block_start, block_end, token_contract_address, ABI):
    """
    Get all ERC-721 transfer events associated with a contract
    Parameters
    ----------
    w3 : 
        connection to mainnet.
    block_start : int
        starting block num to search for events.
    block_end : int
        end block num to search for events.
    token_contract_address : str
        eth address for contract.
    ABI : json
        ABI for contract.
    Returns
    -------
    logs : dict
        dictionary of ERC-721 transfer events.
    """
    
    event_signature=web3.sha3(text="Transfer(address,address,uint256)").hex()
    # Get transfer events
    logs = web3.eth.getLogs({
        "fromBlock": block_start,
        "toBlock": block_end,
        "address": web3.toChecksumAddress(token_contract_address),
        "topics": [event_signature]
    })

    return logs

def decode_event_logs(logs,web3):
    """
    Decode transfer event logs
    
    Parameters
    ----------
    logs - raw event logs
    
    Returns Dictionary
    -------
    indexed by: txHash + tknid
    with: From, to, tokenID, blockNumber, transactionIndex
    """
    event_data = {}
    for i,event in enumerate(logs):    
        
        topic = Web3.toHex(event["topics"][0])
        
        #TRANSFERS
        if topic == web3.sha3(text="Transfer(address,address,uint256)").hex():
        
            from_ = "0x" + Web3.toHex(event["topics"][1])[26:]
            to_   = "0x" + Web3.toHex(event["topics"][2])[26:]
            
            #SuperRare V1 Tokens in Topic
            #SuperRare V2 Tokens in Data
            try:
                tknid = Web3.toInt(event["topics"][3])
            except:
                tknid = int(event["data"],16)
            txhash = event["transactionHash"]
            blocknum = event["blockNumber"]
            log_index = event["logIndex"]
            transaction_index = event["transactionIndex"]    
            event_data[Web3.toInt(txhash)+log_index] = {"txhash":txhash, "from":from_,"to":to_,"tokenID":tknid,'blockNumber':blocknum,'transactionIndex':transaction_index}
        
    return event_data

def get_block_time(web3, block_num):
    """
    Get time of eth block
    Parameters
    ----------
    block_num : int
        ethereum block number.
    Returns
    -------
    blocktime : datetime
        datetime in UTC of block.
    """
    block_info = web3.eth.get_block(block_num)
    blocktime = datetime.fromtimestamp(block_info.timestamp,tz=timezone.utc)
    
    return blocktime

def get_transfer_data(web3, token_contract_address, ABI, block_increment):
    """
    Get Data on ERC-721 Transfers 
    
    Parameters
    ----------
    
    web3 - connection to mainnet 
    token_contract_address - address for platform
    ABI - token's abi
    block_increment - how many blocks to search over
    
    Returns all ERC-721 Transfers decoded 
    -------
    indexed by: txHash + logIndex
    with: From, to, tokenID, blockNumber, transactionIndex
    """
    dfs = []
    current_block = web3.eth.blockNumber
        
    #Get contract
    token_contract_address = Web3.toChecksumAddress(token_contract_address)
    contract = web3.eth.contract(token_contract_address, abi=ABI)
            
    i=0
    while i < current_block:
        block_start = i 
        block_end = i + block_increment
        if block_end > current_block:
            block_end = current_block
        #print(block_start, block_end)
        i += block_increment
         
        #Get transfer events
        logs = get_transfer_events(web3,block_start,block_end,token_contract_address,ABI)
        #Process log
        event_data = decode_event_logs(logs,web3)
        
        #Get dataframe of transfer events
        df_transfers = pd.DataFrame(event_data).transpose()
        dfs.append(df_transfers)

    df_transfers_all = pd.concat(dfs)

    return df_transfers_all

In [ ]:
#here we query every 100,000 blocks
abi                    = json.loads('[{"constant":false,"inputs":[{"name":"_uri","type":"string"},{"name":"_editions","type":"uint256"},{"name":"_salePrice","type":"uint256"}],"name":"addNewTokenWithEditions","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"},{"name":"_salePrice","type":"uint256"}],"name":"setSalePrice","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"_name","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_tokenId","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"currentBidDetailsOfToken","outputs":[{"name":"","type":"uint256"},{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"approvedFor","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"acceptBid","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_creator","type":"address"}],"name":"isWhitelisted","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"bid","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"tokensOf","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_percentage","type":"uint256"}],"name":"setMaintainerPercentage","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_creator","type":"address"}],"name":"whitelistCreator","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"ownerOf","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_uri","type":"string"}],"name":"originalTokenOfUri","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"_symbol","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"cancelBid","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"salePriceOfToken","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_tokenId","type":"uint256"}],"name":"transfer","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"takeOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"_percentage","type":"uint256"}],"name":"setCreatorPercentage","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"tokenURI","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"creatorOfToken","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_tokenId","type":"uint256"}],"name":"buy","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_uri","type":"string"}],"name":"addNewToken","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"creatorPercentage","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"maintainerPercentage","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_creator","type":"address"}],"name":"WhitelistCreator","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"Bid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_seller","type":"address"},{"indexed":false,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"AcceptBid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_bidder","type":"address"},{"indexed":true,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"CancelBid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_buyer","type":"address"},{"indexed":true,"name":"_seller","type":"address"},{"indexed":false,"name":"_amount","type":"uint256"},{"indexed":true,"name":"_tokenId","type":"uint256"}],"name":"Sold","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_tokenId","type":"uint256"},{"indexed":true,"name":"_price","type":"uint256"}],"name":"SalePriceSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"previousOwner","type":"address"},{"indexed":true,"name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_from","type":"address"},{"indexed":true,"name":"_to","type":"address"},{"indexed":false,"name":"_tokenId","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"_owner","type":"address"},{"indexed":true,"name":"_approved","type":"address"},{"indexed":false,"name":"_tokenId","type":"uint256"}],"name":"Approval","type":"event"}]')
token_contract_address = '0x41a322b28d0ff354040e2cbc676f0320d8c8850d'

df_transfers_SR_V1 = get_transfer_data(web3=web3, 
                                       token_contract_address=token_contract_address, 
                                       ABI=abi, 
                                       block_increment=100000)

# Step 3. Finding Robbie-related activity

Now that we have every transfer we can start digging down. First we can start by filtering down to transfer involving Robbie's address, `0x860c4604fe1125ea43f81e613e7afb2aa49546aa`

In [ ]:
df_rb_transfers = df_transfers_SR_V1[(df_transfers_SR_V1["from"] == '0x860c4604fe1125ea43f81e613e7afb2aa49546aa') | (df_transfers_SR_V1["to"] == '0x860c4604fe1125ea43f81e613e7afb2aa49546aa')]

In [ ]:
print(f"There have been {df_rb_transfers.shape[0]} transfers involving Robbie's address.")

In [ ]:
# we need to get block times
blocks = df_rb_transfers.blockNumber.unique()
print(len(blocks))
df_blocktimes = pd.DataFrame()
for block in blocks:
    df_blocktimes.loc[block,"timeUTC"]=get_block_time(web3,block)

In [ ]:
#merge block time on
df_rb_transfers = pd.merge(df_rb_transfers,df_blocktimes,left_on="blockNumber",right_index=True,how='left')

The Lost Robbies were all created on July 16th and July 17th, 2018 right before the Christie's event.

In [ ]:
#filter to july 16 / 17
created_filter = (df_rb_transfers['from'] == '0x0000000000000000000000000000000000000000')
date_filter    = (pd.to_datetime(df_rb_transfers.timeUTC).dt.date.isin([date(2018,7,16),date(2018,7,17)]))
df_lost_robbies_created = df_rb_transfers[created_filter&date_filter]

In [ ]:
df_lost_robbies_created.shape

here are the 300 mints!

In [ ]:
#take a look at the data...
df_lost_robbies_created[["from","to","tokenID","timeUTC","blockNumber"]].reset_index(drop=True).head()

and the tokenIDs...

In [ ]:
#get list of the Lost Robbies tokenIds
lost_robbies_tokenids = list(df_lost_robbies_created.tokenID.unique())

# Step 4 Data Analysis: What's Happened to the Lost Robbies, how many have been claimed?

The easiest way we can tell if the token has been claimed from the blockchain if the token has moved. It's possible some people have access to the private key but just haven't moved them on-chain yet. 

Here, we see how many have only ever moved twice and not claimed. Essentially, minted by Robbie and sent to the gift address but no more than that. 

In [ ]:
df_transfers_SR_V1[df_transfers_SR_V1.tokenID.isin(lost_robbies_tokenids)].groupby("tokenID").tokenID.count().sort_values().value_counts()

### Here is a list of the 31 that have moved more than 2 times (had to have been claimed) and their current owners:

In [ ]:
tokens = df_transfers_SR_V1[df_transfers_SR_V1.tokenID.isin(lost_robbies_tokenids)].groupby("tokenID",as_index=True).tokenID.count().index
values = df_transfers_SR_V1[df_transfers_SR_V1.tokenID.isin(lost_robbies_tokenids)].groupby("tokenID",as_index=True).tokenID.count().values

In [ ]:
metadata['name'][-3:]

In [ ]:
df_owners = pd.DataFrame()
for tokenID,times_moved in zip(tokens,values):
    if times_moved>2:
        token_uri = get_tokenURI(tokenID)
        metadata  = get_metadata(token_uri)
        print(f"{metadata['name']} has been claimed (token {tokenID}), and is currently owned by {contract.functions.ownerOf(tokenID).call()}")
        string = f"{metadata['name']} has been claimed (token {tokenID}), and is currently owned by {contract.functions.ownerOf(tokenID).call()}"
        #print("____________")
        #print(f"https://superrare.com/artwork/ai-generated-nude-portrait-7-frame-{metadata['name'][-3:]}-{tokenID}")
        df_owners.loc[tokenID,"Owner"] = string

Another way we can check is if the address has sent transactions before or holds a positive balance of eth (the addresses with 0 txs are very likely dormant). For example, [AI Generated Nude Portrait #7 Frame #8](https://superrare.com/artwork/ai-generated-nude-portrait-7-frame-8-198) (token 198) has been claimed by SR user Christensen, but they have not moved it from the address created by SR and Artnome for the conference. It holds a positive balance of ETH, which is needed to pay gas fees for interacting with the smart contract and setting list prices.

In [ ]:
# here we get the current eth balance for each of the 269 that have only moved twice.

#get all transfers involving Lost Robbies
lost_robbie_transfers = df_transfers_SR_V1[df_transfers_SR_V1.tokenID.isin(lost_robbies_tokenids)].copy()

#only those tokens still owned by OG address
filter_2 = (lost_robbie_transfers.groupby('tokenID').tokenID.transform('count')==2)
lost_robbie_transfers_tokens_orig_adr = lost_robbie_transfers[filter_2].copy()

df_eth_bal_lost_robbie_tokens = pd.DataFrame()
for token in lost_robbie_transfers_tokens_orig_adr.tokenID.unique():
    #get current owner
    owner   = contract.functions.ownerOf(token).call()
    #get eth balance (in wei)
    eth_bal = web3.eth.get_balance(owner)/1e18
    #get tx count
    tx_cnt = web3.eth.get_transaction_count(owner)
    
    #append to df
    df_eth_bal_lost_robbie_tokens.loc[token,"eth_bal"]=eth_bal
    df_eth_bal_lost_robbie_tokens.loc[token,"owner"]  =owner
    df_eth_bal_lost_robbie_tokens.loc[token,"tx_cnt"] =tx_cnt

In [ ]:
df_eth_bal_lost_robbie_tokens[(df_eth_bal_lost_robbie_tokens.eth_bal>0)|(df_eth_bal_lost_robbie_tokens.tx_cnt>0)]

From this we have an additional 5 tokens that either have a non-zero eth balance or have made at least one tx. 

To double check, we can do some manual checks on the SuperRare front end to see if these addresses actually have active SuperRare users associated with them. 

- `0x9Ced77beed0381B5Ea7d2aE6b859B77Ae0FF0F2D` = SR user [@christensen](https://superrare.com/christensen) so this is claimed
- `0xb435bf979E666a2Fee83e52d3ebce375A111dd7A` does not have an associated SR user so this is very likely unclaimed and it has 0 tx (the positive eth could have just been a test tx to the address)
- `0xCa96436021bE7AF640b4D6cf0bDB92D47051dC66` = SR user [@nikoknits](https://superrare.com/nikoknits) so this is claimed
- `0xAE972C42a681C2Ba4DF60f679b6Def13A970931d` = SR user [@tilmankriesel](https://superrare.com/tilmankriesel) so this is claimed
- `0xa63f0ddc7216258006dc892BFd422eDdFE297F89` = SR user [@cxrxhx](https://superrare.com/cxrxhx) so this is also claimed

So in total, we had 4 additional Lost Robbies that have been claimed, bringing the total to 35 of 300.

In [ ]:
#print owners and names of the works
for tokenID in df_eth_bal_lost_robbie_tokens[df_eth_bal_lost_robbie_tokens.eth_bal>0].index:
    token_uri = get_tokenURI(tokenID)
    metadata  = get_metadata(token_uri)
    print(f"{metadata['name']}(token {tokenID}) is currently owned by {contract.functions.ownerOf(tokenID).call()}")
    print("____________")

Finally, there is AI Generated Nude Portrait #7 Frame #64 (token 254) which was recently auctioned off at Sotheby's and thus has an owner and is not lost. 

In [ ]:
token_uri = get_tokenURI(254)
metadata  = get_metadata(token_uri)
print(metadata)

In [ ]:
#moved onchain
filtermoved = (lost_robbie_transfers.groupby('tokenID').tokenID.transform('count')>2)
lost_robbie_transferred = lost_robbie_transfers[filtermoved].copy()
moved_onchain = list(lost_robbie_transferred.tokenID.unique())
#sothebys
sothebys = [254]
#owned by original address
og_address = [198,238,309,415]

claimed_tkns = og_address + moved_onchain + sothebys

So as of March 3, 2022, 264 are likely lost, with only 36 being claimed/i.e. an owner with control of the private key to the address.

A Sankey graph is a nice way to visualize this:

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 20,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["300 Total", "36 Claimed", "264 Likely Lost","31 Changed Owners/Moved On-Chain","5 Held by Original Address"],
      color = ["navy","darkgreen","maroon","green","green"]
    ),
    link = dict(
                source = [0, 0,      # Start
                          1, 1],      # Moved?
                target = [1, 2, 3, 4, 5],
                value = [36,264,31,5]
  ))])

fig.update_layout(title_text="How Many of the Lost Robbies were Claimed?", font_size=20)
fig.show()

### What was the most recent to move out of the original address?

In [ ]:
#find all of the NFTs that have moved more than twice 
filter_more_than2 = (lost_robbie_transfers.groupby('tokenID').tokenID.transform('count')>2)
lost_robbie_transfers_tokens_claimed = lost_robbie_transfers[filter_more_than2].copy()
#sort on tokenID and block height
lost_robbie_transfers_tokens_claimed.sort_values(["tokenID","blockNumber"],inplace=True)
#assign transfer number per token
lost_robbie_transfers_tokens_claimed["blockNumber"] = lost_robbie_transfers_tokens_claimed["blockNumber"].astype(int)
lost_robbie_transfers_tokens_claimed["transfer_number"] = lost_robbie_transfers_tokens_claimed.groupby("tokenID")["blockNumber"].rank("dense",ascending=True)
#of the 3rd transfers, what was most recent (highest block height?)
third_transfers = lost_robbie_transfers_tokens_claimed[lost_robbie_transfers_tokens_claimed['transfer_number']==3]
most_recent = third_transfers.loc[third_transfers['blockNumber'].idxmax()]

print(f"The most recent Lost Robbie to be claimed was {get_metadata(get_tokenURI(most_recent.tokenID))['name']} (token {most_recent.tokenID}) on {get_block_time(web3,int(most_recent.blockNumber))}")

Finally, we can collect all of the lost robbies easily and create a collage.

In [ ]:
def create_lost_robbies_collage(lost_robbies_tokenids,claimed_tkns):
    
    #create image
    count = 0
    all_arrays=[]
    arrays = []
    total = 0

    #Get NFTs with that tag
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    for tkn in lost_robbies_tokenids:

        src = get_tokenURI(tkn)
        meta = get_metadata(src)
        i = load_image(meta)
        #display(i)''
        #print(tkn)
        i = i.resize((100,100))
        
        #if it is claimed, make a border around it
        if tkn in claimed_tkns:
            print("claimed")
            i = ImageOps.expand(i,border=(5,5),fill='yellow')
        
        i = i.resize((100,100))
        #print("____________")
        _array = img_to_array(i)
        #print(_array.shape)
        if _array.shape == (100,100,3):
            _array = np.resize(_array, (100,100,3))

            count+=1
            total +=1
            arrays.append(_array)

            if count == 20:
                concatenated = np.concatenate(arrays, axis=1)
                all_arrays.append(concatenated)
                arrays = []
                count = 0
                #print(len(arrays))

    big_image = np.concatenate(all_arrays,axis=0)
    big_im = array_to_img(big_image)
    return big_im

In [ ]:
create_lost_robbies_collage(lost_robbies_tokenids,claimed_tkns)